In [ ]:
#Read the data files
import pandas as pd

total_commits=pd.read_csv('/home/kc/Projects/data_files/vscode.csv')
#total_commits.shape

In [ ]:
#Creating various features for each modification

total_commits['total_changed'] = total_commits['lines_added']+total_commits['lines_removed']
total_commits['size'].loc[total_commits['size'] == 0] = total_commits['total_changed']
total_commits['ratio_changed'] = total_commits['total_changed']/total_commits['size']
total_commits['rated_complexity'] = total_commits['ratio_changed']*total_commits['complexity']
total_commits['total_dmm_size'] = total_commits['total_changed']*total_commits['dmm_unit_size']
total_commits['total_dmm_unit_complexity'] = total_commits['total_changed']*total_commits['dmm_unit_complexity']
total_commits['total_dmm_unit_interfacing'] = total_commits['total_changed']*total_commits['dmm_unit_interfacing']
total_commits['scaled_rated_complexity']=total_commits['rated_complexity']*total_commits['no._of_mod_files']

In [ ]:
# Preprocessing the data. ML requires the data to be converted to numericals
#ml_commits = total_commits[['hash','Author', 'no._of_mod_files', 'dmm_unit_size',
#       'dmm_unit_complexity', 'dmm_unit_interfacing', 'complexity', 'functions', 'lines_added', 'lines_removed', 
#       'tokens', 'type']]

ml_commits = total_commits[['hash','Author','total_changed','ratio_changed','rated_complexity',
                            'total_dmm_unit_complexity','total_dmm_size','total_dmm_unit_interfacing', 'scaled_rated_complexity']]

# Resetting the frame's index. It is required to retain the integrity of the frame
ml_commits = ml_commits.reset_index().drop(columns = 'index')

# Temporarily dropping text columns for numeric processing
ml_commits_noText = ml_commits.drop(columns = ['Author','hash'])

# Explicitely converting fields to numeric types and filling the NaNs with zeros
ml_commits_numeric = ml_commits_noText.apply(pd.to_numeric,errors ='coerce').fillna(0)

# Adding the Author column back to create a 'total' data frame
ml_commits_total = ml_commits_numeric.copy()
ml_commits_total['Author'] = ml_commits['Author']
ml_commits_total['hash'] = ml_commits['hash']

In [ ]:
# Remove outliers. 
from scipy import stats
import numpy as np

# Calculate z_scores (and if zscore is greater than '3', then its an outlier) and collect normal subset.
ml_commits_nout = ml_commits_total[(np.abs(stats.zscore(ml_commits_total.select_dtypes(exclude='object'))) < 3).all(axis=1)]
ml_commits_nout.to_csv('/home/kc/junk/vscode_nout.csv')

# Collect outliers
ml_commits_out = ml_commits_total[~(np.abs(stats.zscore(ml_commits_total.select_dtypes(exclude='object'))) < 3).all(axis=1)]
ml_commits_out.to_csv('/home/kc/junk/vscode_out.csv')

In [ ]:
# Applying scaler to regular data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler

# Use minMax scaler since this does not distort
# https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02
scaler = MinMaxScaler()
ml_commits_nout_numeric = ml_commits_nout.drop(columns = ['Author','hash'])
data_scaled = scaler.fit_transform(ml_commits_nout_numeric)

In [ ]:
# Try and figure out how many clusters are optimum
# Plot the inertia curve to find the 'elbow'
from sklearn.cluster import KMeans

SSE = []

for cluster in range(1,20):
    kmeans_ss = KMeans(n_clusters = cluster, init='k-means++', random_state = 42)
    kmeans_ss.fit(data_scaled)
    SSE.append(kmeans_ss.inertia_)

    # converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,20), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

In [ ]:
# Instantiating the KMeans object based on number discovered above
k = 5
kmeans = KMeans(n_clusters = k, init='k-means++', max_iter = 20, random_state = 42)

# Creating the model by passing our non-outlier data to Kmeans object
trained_model = kmeans.fit(data_scaled)

# Retrieving centroids
centroids = kmeans.cluster_centers_

# Labels of KMeans clusters change for every iteration. 
# We need to preserve these lables for runs with multiple/incremental input data sets.
# Creating a hack to attempt to preserve their identities.
# Calculating the arithmetic sum of all values in each centroid. This is done to fix labels for each training iteration
# Assumption is that bigger the values in a centroid, higher the original feature values. This assumption is true only for 
#        data set and our features. We need to definitely confirm/verify this assumption.
combinedCentroids = centroids[trained_model.labels_].sum(axis=1)

# adding column with combined centroid values to the original dataframe 
ml_commits_nout['center'] = combinedCentroids
#print(combinedCentroids)

# Creating a dictionary with combined centroid values and target cluster labels
unique_centroids = np.unique(combinedCentroids).tolist()
cluster_labels = np.arange(k).tolist()
cluster_dict = dict(zip(unique_centroids,cluster_labels))
#print(g)
ml_commits_nout['fixed_cluster'] = ml_commits_nout['center'].map(cluster_dict)
ml_commits_nout

In [ ]:
ml_clustered = ml_commits_nout.groupby(['fixed_cluster'],as_index=True).count()
ml_clustered['hash']
ml_commits_nout[ml_commits_nout['fixed_cluster']==4].head(5)
ml_commits_nout.to_csv('/home/kc/junk/react_clusters_202007212010.csv')